In [1]:
import os,cv2,torch,torchvision
import detectron2
from detectron2.evaluation import COCOEvaluator,inference_context,inference_on_dataset
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog,MetadataCatalog,build_detection_test_loader
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.model_zoo import get_checkpoint_url,get_config_file
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import Visualizer,BoxMode
import numpy as np
import pandas as pd
from tqdm import tqdm
from xml.etree import ElementTree

In [2]:
imgs = []
x_mins = []
y_mins = []
y_maxs = []
x_maxs = []

In [3]:
for img in zip(os.listdir('./data/')):
    try:
        img = img[0]
        tree = ElementTree.parse(f'./data/{img}')
        root = tree.getroot()
        for box in root.findall('.//bndbox'):
            xmin = box.find('xmin').text
            xmax = box.find('xmax').text
            ymin = box.find('ymin').text
            ymax = box.find('ymax').text
        imgs.append('./data/' + img.replace('.xml','.jpg'))
        x_mins.append(int(xmin))
        x_maxs.append(int(xmax))
        y_mins.append(int(ymin))
        y_maxs.append(int(ymax))
    except Exception as e:
        pass

In [4]:
data = pd.DataFrame({'frame':imgs,'xmin':x_mins,'xmax':x_maxs,'ymax':y_maxs,'ymin':y_mins})

In [5]:
def load_data(data=data):
    dataset = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        xmin,ymin,xmax,ymax = info['xmin'],info['ymin'],info['xmax'],info['ymax']
        height,width = cv2.imread(info['frame']).shape[:2]
        record['file_name'] = info['frame']
        record['height'] = height
        record['width'] = width
        info['class_id'] = 0
        record['cateogry_id'] = 0
        objs = [{'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymax']],'bbox_mode':BoxMode.XYXY_ABS,'iscrowd':0,'category_id':0}]
        record['image_id'] = idx
        record['annotations'] = objs
        record['class_id'] = 0
        record['xmin'] = xmin
        record['ymin'] = ymin
        record['xmax'] = xmax
        record['ymax'] = ymax
        dataset.append(record)
    return dataset

In [6]:
DatasetCatalog.register('data',lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=['PathHole'])
metadata = MetadataCatalog.get('data')

In [7]:
import wandb


In [8]:
import matplotlib.pyplot as plt


In [9]:
# torch.cuda.empty_cache()
# wandb.init(sync_tensorboard=True,name='baseline')
# model = 'COCO-Detection/faster_rcnn_R_50_C4_1x.yaml'
# cfg = get_cfg()
# cfg.merge_from_file(get_config_file(model))
# cfg.DATASETS.TRAIN = ('data',)
# cfg.DATASETS.TEST = ()
# cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
# cfg.SOLVER.MAX_ITER = 2500 + 1250
# cfg.SOLVER.BASE_LR = 0.00025
# cfg.SOLVER.STEPS = []
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# img = cv2.imread('./data/img-1.jpg')
# predictor = DefaultPredictor(cfg)
# preds = predictor(img)['instances'].to('cpu')
# v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
# v = v.draw_instance_predictions(preds)
# v = v.get_image()[:,:,::-1]
# wandb.log({'img':wandb.Image(v)})
# plt.figure(figsize=(10,7))
# plt.imshow(v)
# plt.show()

In [10]:
# from detectron2.checkpoint import DetectionCheckpointer
# checkpointer = DetectionCheckpointer(model, save_dir="output")
# checkpointer.save("model_999") 

In [11]:
models = [
    'COCO-Detection/faster_rcnn_R_101_C4_3x.yaml',
    'COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml',
    'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml',
    'COCO-Detection/faster_rcnn_R_50_C4_1x.yaml',
    'COCO-Detection/faster_rcnn_R_50_C4_3x.yaml',
    'COCO-Detection/faster_rcnn_R_50_DC5_1x.yaml',
    'COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml',
    'COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml',
    'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml',
    'COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml',
    'COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml'
]

In [12]:
BASE_LRS = [
    0.00025,
    0.00050,
    0.00075,
    0.0001,
    0.001,
    0.01,
    0.0002,
    0.0003,
    0.0004,
    0.0005,
    0.0006,
    0.0007,
    0.0008,
    0.0009,
]

In [13]:
IMS_PER_BATCHS = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10
]

In [14]:
BATCH_SIZE_PER_IMAGES = [
    121,
    122,
    123,
    124,
    125,
    126,
    127,
    129
]

In [15]:
# for model in models:
#     try:
#         torch.cuda.empty_cache()
#         wandb.init(sync_tensorboard=True,name=model)
#         model = model
#         cfg = get_cfg()
#         cfg.merge_from_file(get_config_file(model))
#         cfg.DATASETS.TRAIN = ('data',)
#         cfg.DATASETS.TEST = ()
#         cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
#         cfg.SOLVER.MAX_ITER = 2500 + 1250
#         cfg.SOLVER.BASE_LR = 0.00025
#         cfg.SOLVER.STEPS = []
#         cfg.SOLVER.IMS_PER_BATCH = 2
#         cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
#         cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
#         trainer = DefaultTrainer(cfg)
#         trainer.resume_or_load(resume=False)
#         trainer.train()
#         cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
#         cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#         img = cv2.imread('./data/img-1.jpg')
#         predictor = DefaultPredictor(cfg)
#         preds = predictor(img)['instances'].to('cpu')
#         v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
#         v = v.draw_instance_predictions(preds)
#         v = v.get_image()[:,:,::-1]
#         wandb.log({'img':wandb.Image(v)})
#         plt.figure(figsize=(10,7))
#         plt.imshow(v)
#         plt.show()
#     except:
#         pass

In [16]:
model = 'COCO-Detection/faster_rcnn_R_101_C4_3x.yaml'

In [ ]:
# for BASE_LR in BASE_LRS:
#     torch.cuda.empty_cache()
#     wandb.init(sync_tensorboard=True,name=f'{BASE_LR}')
#     cfg = get_cfg()
#     cfg.merge_from_file(get_config_file(model))
#     cfg.DATASETS.TRAIN = ('data',)
#     cfg.DATASETS.TEST = ()
#     cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
#     cfg.SOLVER.MAX_ITER = 2500 + 1250
#     cfg.SOLVER.BASE_LR = BASE_LR
#     cfg.SOLVER.STEPS = []
#     cfg.SOLVER.IMS_PER_BATCH = 2
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
#     cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
#     trainer = DefaultTrainer(cfg)
#     trainer.resume_or_load(resume=False)
#     trainer.train()
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#     img = cv2.imread('./data/img-1.jpg')
#     predictor = DefaultPredictor(cfg)
#     preds = predictor(img)['instances'].to('cpu')
#     v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
#     v = v.draw_instance_predictions(preds)
#     v = v.get_image()[:,:,::-1]
#     wandb.log({'img':wandb.Image(v)})
#     plt.figure(figsize=(10,7))
#     plt.imshow(v)
#     plt.show()

In [ ]:
# for IMS_PER_BATCH in IMS_PER_BATCHS:
#     torch.cuda.empty_cache()
#     wandb.init(sync_tensorboard=True,name=model)
#     model = 
#     cfg = get_cfg()
#     cfg.merge_from_file(get_config_file(model))
#     cfg.DATASETS.TRAIN = ('data',)
#     cfg.DATASETS.TEST = ()
#     cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
#     cfg.SOLVER.MAX_ITER = 2500 + 1250
#     cfg.SOLVER.BASE_LR = 
#     cfg.SOLVER.STEPS = []
#     cfg.SOLVER.IMS_PER_BATCH = IMS_PER_BATCH
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
#     cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
#     trainer = DefaultTrainer(cfg)
#     trainer.resume_or_load(resume=False)
#     trainer.train()
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#     img = cv2.imread('./data/img-1.jpg')
#     predictor = DefaultPredictor(cfg)
#     preds = predictor(img)['instances'].to('cpu')
#     v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
#     v = v.draw_instance_predictions(preds)
#     v = v.get_image()[:,:,::-1]
#     wandb.log({'img':wandb.Image(v)})
#     plt.figure(figsize=(10,7))
#     plt.imshow(v)
#     plt.show()

In [ ]:
# for BATCH_SIZE_PER_IMAGE in BATCH_SIZE_PER_IMAGES:
#     torch.cuda.empty_cache()
#     wandb.init(sync_tensorboard=True,name=model)
#     model = 
#     cfg = get_cfg()
#     cfg.merge_from_file(get_config_file(model))
#     cfg.DATASETS.TRAIN = ('data',)
#     cfg.DATASETS.TEST = ()
#     cfg.MODEL.WEIGHTS = get_checkpoint_url(model)
#     cfg.SOLVER.MAX_ITER = 2500 + 1250
#     cfg.SOLVER.BASE_LR = 
#     cfg.SOLVER.STEPS = []
#     cfg.SOLVER.IMS_PER_BATCH = IMS_PER_BATCH
#     cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(['PathHole'])
#     cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = BATCH_SIZE_PER_IMAGE
#     trainer = DefaultTrainer(cfg)
#     trainer.resume_or_load(resume=False)
#     trainer.train()
#     cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.475
#     cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#     img = cv2.imread('./data/img-1.jpg')
#     predictor = DefaultPredictor(cfg)
#     preds = predictor(img)['instances'].to('cpu')
#     v = Visualizer(img[:,:,::-1],metadata=metadata,scale=1)
#     v = v.draw_instance_predictions(preds)
#     v = v.get_image()[:,:,::-1]
#     wandb.log({'img':wandb.Image(v)})
#     plt.figure(figsize=(10,7))
#     plt.imshow(v)
#     plt.show()